In [126]:
import pandas as pd
import numpy as np

In [127]:
city_pop = pd.read_csv("sub-est2021_all.csv",encoding='latin-1')
state_codes = pd.read_csv("state_codes.csv")

city_pop = city_pop[["PLACE","NAME", "STNAME","POPESTIMATE2021"]] 
city_only = [i.lower().__contains__("city") for i in city_pop["NAME"]] 
city_pop = city_pop[city_only]
city_pop = city_pop[city_pop["POPESTIMATE2021"]>=50000]
city_pop = city_pop.set_axis(['place_code','city', 'state', 'pop_estimate_2021'], axis='columns', copy = False)
city_pop["city"] = [i.replace(" city","") for i in city_pop["city"]]

state_codes = state_codes[["state","code"]].set_axis(['state', 'state_code'], axis='columns', copy = False)
city_pop = pd.merge(city_pop,state_codes, on = "state", how = "left")
city_pop["city_state"] = city_pop["city"] + ", " + city_pop["state"]
city_pop["city_state_code"] = city_pop["city"] + ", " + city_pop["state_code"]
city_pop["place_state_code"] = [str(i)+j for i,j in zip(city_pop["place_code"],city_pop["state_code"])]
city_pop = city_pop.drop_duplicates(subset = "place_state_code")
city_pop = city_pop.drop_duplicates(subset = "city_state")
list(city_pop)
city_pop #760

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL
...,...,...,...,...,...,...,...,...
1833,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI
1834,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI
1835,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI
1872,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY


In [128]:
city_land = pd.read_csv("2020_Gaz_place_national.txt",sep = "\t",encoding='latin-1')
state_codes = pd.read_csv("state_codes.csv")
city_land
city_only = [i.lower().__contains__("city") for i in city_land["NAME"]]
city_land= city_land[city_only]
city_land = city_land[["USPS","GEOID","NAME","ALAND"]].set_axis(['state_code', 'geo_code', 'city', 'land_area_sqm'], axis='columns', copy = False)

state_codes = state_codes[["state","code"]].set_axis(['state', 'state_code'], axis='columns', copy = False)
city_land = pd.merge(city_land, state_codes, on = "state_code", how = "left")
city_land["city"] = [i.replace(" city","") for i in city_land["city"]]
city_land["city_state"] = city_land["city"] + ", " + city_land["state"]
city_land["city_state_code"] = city_land["city"] + ", " + city_land["state_code"]
city_land["land_area_sqkm"] = city_land["land_area_sqm"]*0.000001
city_land["geo_code"] = [str(i) for i in city_land["geo_code"]]
# city_land #10512

In [129]:
walking_data = pd.merge(city_pop,city_land[["city_state", "geo_code", "land_area_sqkm"]], on = "city_state", how = "left")
walking_data["pop_per_km2"] = walking_data["pop_estimate_2021"]/walking_data["land_area_sqkm"]
walking_data #760

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,pop_per_km2
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,500.137630
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,518.840515
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,410.581694
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,305.930918
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,743.742746
...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,1924.323130
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,1077.955667
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,2017.954722
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,853.090863


In [130]:
county = pd.read_csv("city_county.csv")
county["county_state"] = list(pd.Series([i.replace(" County","") for i in county["county"]]) + ", " + county["state"])
walking_data = pd.merge(walking_data, county[["county_state","city_state_code"]], on = "city_state_code", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,pop_per_km2,county_state
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,500.137630,"Lee, Alabama"
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,518.840515,"Jefferson, Alabama"
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,410.581694,"Morgan, Alabama"
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,305.930918,"Houston, Alabama"
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,743.742746,"Jefferson, Alabama"
...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,1924.323130,"Racine, Wisconsin"
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,1077.955667,"Waukesha, Wisconsin"
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,2017.954722,"Milwaukee, Wisconsin"
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,853.090863,"Natrona, Wyoming"


In [131]:
age_sex_columns = pd.read_csv("Age_and_Sex_2021-Column-Metadata.csv",encoding='latin-1',low_memory=False)
age_sex = pd.read_csv("Age_and_Sex_2021-Data.csv",encoding='latin-1')
age_sex = age_sex.drop_duplicates(subset='GEO_ID')
age_sex
age_sex_col = ["NAME", "S0101_C01_032E","S0101_C03_001E","S0101_C05_001E"]
age_sex_label = ["city_state","median_age", "male","female"]

age_sex["NAME"] = [i.replace(" city", "") for i in age_sex["NAME"]]

age_sex = age_sex[age_sex_col].set_axis(age_sex_label, axis='columns', copy = False)



age_sex["male"] = age_sex["male"].replace("N", float("nan"))
age_sex["male"] = [float(i) for i in age_sex["male"]]


age_sex["female"] = age_sex["female"].replace("N", float("nan"))
age_sex["female"] = [float(i) for i in age_sex["female"]]

walking_data = pd.merge(walking_data, age_sex, on = "city_state", how = "left")

walking_data["male"] = walking_data["male"]/walking_data["pop_estimate_2021"]
walking_data["female"] = walking_data["female"]/walking_data["pop_estimate_2021"]

walking_data


C:\Users\chane\AppData\Local\Temp\ipykernel_9128\986331991.py:2: DtypeWarning: Columns (2,5,6,7,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,89,90,91,92,93,94,95,96,97,102,103,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,158,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,242,243,244,245,246,247,248,249,250,251,252,253,254,255,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,306,307,308,309,310,311,312,313,314,31

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,pop_per_km2,county_state,median_age,male,female
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,500.137630,"Lee, Alabama",27.8,0.483186,0.516662
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,518.840515,"Jefferson, Alabama",37.2,0.479291,0.514812
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,410.581694,"Morgan, Alabama",NaN,NaN,NaN
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,305.930918,"Houston, Alabama",40.5,0.473565,0.527952
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,743.742746,"Jefferson, Alabama",38.9,0.504066,0.495923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,1924.323130,"Racine, Wisconsin",34.6,0.476954,0.523098
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,1077.955667,"Waukesha, Wisconsin",36.2,0.482654,0.517318
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,2017.954722,"Milwaukee, Wisconsin",NaN,NaN,NaN
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,853.090863,"Natrona, Wyoming",NaN,NaN,NaN


In [132]:
age_sex

,city_state,median_age,male,female
0,"Auburn, Alabama",27.8,37961.0,40591.0
2,"Birmingham, Alabama",37.2,94696.0,101714.0
4,"Dothan, Alabama",40.5,33706.0,37577.0
6,"Hoover, Alabama",38.9,46671.0,45917.0
8,"Huntsville, Alabama",38.0,106904.0,108578.0
...,...,...,...,...
1258,"Caguas zona urbana, Puerto Rico",44.2,34275.0,42302.0
1260,"Carolina zona urbana, Puerto Rico",43.4,63819.0,77231.0
1262,"Guaynabo zona urbana, Puerto Rico",46.9,32710.0,39378.0
1264,"Ponce zona urbana, Puerto Rico",43.8,52732.0,59273.0


In [133]:
race = pd.read_csv("Race_2020.p1-Data.csv",encoding='latin-1')
race_columns = pd.read_csv("Race_2020.p1-Column-Metadata.csv",encoding='latin-1')
race
race_columns = race_columns[[ not i.__contains__("A") for i in race_columns["Column Name"]]]
more_col = pd.Series(["P1_009N"])
more_label = ["two_or_more_races"]
temp = race_columns[[i.__contains__("alone") for i in race_columns["Label"]]]
alone_col = temp["Column Name"]
alone_label = [i.split("!!")[-1].lower().replace(" ","_") for i in temp["Label"]]

col = pd.concat([pd.Series("NAME"),alone_col,more_col])
label = ["city_state"] + alone_label + more_label

race = race[[i.__contains__("city") for i in race["NAME"]]]
race["NAME"] = [i.replace(" city", "") for i in race["NAME"]]
race = race[col].set_axis(label,axis="columns",copy=False)

for val in alone_label + more_label:
    race[val] = race[val].replace("N", float("nan"))
    race[val] = [float(i) for i in race[val]]

walking_data = pd.merge(walking_data, race, on = "city_state", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,median_age,male,female,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,two_or_more_races
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,27.8,0.483186,0.516662,48918.0,14009.0,238.0,7414.0,38.0,1597.0,3929.0
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,37.2,0.479291,0.514812,47209.0,137296.0,755.0,3275.0,123.0,5360.0,6715.0
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,NaN,NaN,NaN,33190.0,13441.0,433.0,531.0,56.0,5983.0,4304.0
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,40.5,0.473565,0.527952,40496.0,23896.0,285.0,1141.0,47.0,1444.0,3763.0
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,38.9,0.504066,0.495923,63664.0,15621.0,254.0,5929.0,20.0,1940.0,5178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,34.6,0.476954,0.523098,40048.0,18618.0,547.0,601.0,23.0,8077.0,9902.0
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,36.2,0.482654,0.517318,55908.0,2466.0,373.0,2404.0,41.0,3162.0,6804.0
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,NaN,NaN,NaN,44286.0,3818.0,787.0,1545.0,23.0,3615.0,6251.0
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,NaN,NaN,NaN,50857.0,672.0,804.0,531.0,59.0,1653.0,4462.0


In [134]:
hisp = pd.read_csv("Hispanic_2020.P2-Data.csv",encoding='latin-1', low_memory=False)
hisp_col = ["NAME","P2_002N"]
hisp_label = ["city_state","hispanic_or_latino"]
hisp = hisp[[i.__contains__("city") for i in hisp["NAME"]]]
hisp["NAME"] = [i.replace(" city","") for i in hisp["NAME"]]
hisp = hisp[hisp_col].set_axis(hisp_label, axis="columns", copy=False)

hisp["hispanic_or_latino"] = hisp["hispanic_or_latino"].replace("N", float("nan"))
hisp["hispanic_or_latino"] = [float(i) for i in hisp["hispanic_or_latino"]]

walking_data = pd.merge(walking_data, hisp, on = "city_state", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,male,female,white_alone,black_or_african_american_alone,american_indian_and_alaska_native_alone,asian_alone,native_hawaiian_and_other_pacific_islander_alone,some_other_race_alone,two_or_more_races,hispanic_or_latino
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,0.483186,0.516662,48918.0,14009.0,238.0,7414.0,38.0,1597.0,3929.0,3619.0
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,0.479291,0.514812,47209.0,137296.0,755.0,3275.0,123.0,5360.0,6715.0,9274.0
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,NaN,NaN,33190.0,13441.0,433.0,531.0,56.0,5983.0,4304.0,9070.0
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,0.473565,0.527952,40496.0,23896.0,285.0,1141.0,47.0,1444.0,3763.0,3131.0
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,0.504066,0.495923,63664.0,15621.0,254.0,5929.0,20.0,1940.0,5178.0,4704.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,0.476954,0.523098,40048.0,18618.0,547.0,601.0,23.0,8077.0,9902.0,18856.0
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,0.482654,0.517318,55908.0,2466.0,373.0,2404.0,41.0,3162.0,6804.0,9979.0
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,NaN,NaN,44286.0,3818.0,787.0,1545.0,23.0,3615.0,6251.0,10211.0
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,NaN,NaN,50857.0,672.0,804.0,531.0,59.0,1653.0,4462.0,5582.0


In [135]:
race_etc = ["white_alone",
"black_or_african_american_alone",
"american_indian_and_alaska_native_alone",
"asian_alone",
"native_hawaiian_and_other_pacific_islander_alone",
"some_other_race_alone",
"two_or_more_races",
"hispanic_or_latino"
]
for etc in race_etc:
    walking_data[etc] = walking_data[etc]/walking_data["pop_estimate_2021"]

In [136]:
health = pd.read_csv("PLACES__Local_Data_for_Better_Health__County_Data_2022_release.csv",encoding='latin-1')
health["MeasureId"] = [i.lower() for i in health["MeasureId"]]
health[["Measure", "MeasureId"]].drop_duplicates().to_csv("health_variable_key.csv",index = False)

health["county_state"] =  health["LocationName"] + ", " + health["StateDesc"]
health = health[["county_state","MeasureId","Data_Value"]]

health = health.pivot_table(values='Data_Value', index=["county_state"], columns="MeasureId")

walking_data = pd.merge(walking_data, health, on = "county_state", how = "left")
walking_data




,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,highchol,kidney,lpa,mammouse,mhlth,obesity,phlth,sleep,stroke,teethlost
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,31.80,2.90,24.30,74.10,16.00,36.00,10.60,35.85,3.10,11.85
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,30.25,3.20,25.10,80.55,15.15,38.85,10.35,37.70,3.55,12.10
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,33.80,3.20,28.60,73.80,16.80,40.15,12.25,37.85,3.60,14.00
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,33.95,3.40,32.65,71.25,16.85,39.10,12.30,39.90,3.95,14.20
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,30.25,3.20,25.10,80.55,15.15,38.85,10.35,37.70,3.55,12.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,32.55,2.90,21.70,76.00,13.70,36.10,10.15,32.90,3.00,10.20
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,30.60,2.45,16.25,76.70,11.70,28.95,7.70,28.15,2.30,6.60
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,31.10,3.05,23.35,76.30,14.05,37.65,10.20,32.70,3.15,11.20
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,28.90,2.60,25.35,62.00,13.25,34.10,9.35,33.30,2.65,12.90


In [137]:
# covid = pd.read_csv("epidemiology.csv",encoding='latin-1')
# covid = covid[[str(i)[0:2]=="US" for i in covid["location_key"]]]
# covid.to_csv("covid_US.csv", index = False)

covid = pd.read_csv("covid_US.csv",encoding='latin-1')

# covid = covid.dropna(subset=["cumulative_confirmed","cumulative_deceased"])
# #covid.pivot_table(values=["date","cumulative_confirmed","cumulative_deceased"], index="location_key",aggfunc={"date":np.max,"cumulative_confirmed":np.max,"cumulative_deceased": np.max})
covid = covid[[i=="2022-05-13" for i in covid["date"]]]
covid = covid[["date","location_key","cumulative_confirmed","cumulative_deceased"]]

covid_index = pd.read_csv("covid_index.csv",encoding='latin-1')
covid_index = covid_index[covid_index["country_code"]=="US"]
covid_index["county_state"] = list(pd.Series([str(county).replace(" County","") for county in covid_index["subregion2_name"]])+ ", "+ pd.Series([str(state) for state in covid_index["subregion1_name"]]))

covid = pd.merge(covid,covid_index[["county_state", "location_key"]],on = "location_key", how = "left")

covid = covid.drop_duplicates(subset="county_state")

#turn into ratios
sub_county = walking_data[["county_state","pop_estimate_2021"]].groupby("county_state").sum()
temp = pd.merge(covid, sub_county, on = "county_state", how = "left")
temp["cumulative_confirmed"] = [cc/pop for cc, pop in zip(temp["cumulative_confirmed"], temp["pop_estimate_2021"])]
temp["cumulative_deceased"] = [cd/pop for cd, pop in zip(temp["cumulative_deceased"], temp["pop_estimate_2021"])]

walking_data = pd.merge(walking_data, temp[["county_state","cumulative_confirmed","cumulative_deceased"]], on = "county_state", how = "left")




In [138]:
# # webscrape for walkability data
# import requests
# import re
# # Making a GET request
# n = len(walking_data)
# state_code = walking_data["state_code"]
# city = walking_data["city"]
# urls = ["".join(["https://www.walkscore.com", "/", state_code[idx], "/", city[idx].replace(" ","_")]) for idx in range(n)]
# all_data = [str(requests.get(url).content) for url in urls]

# walk_score = []
# bike_score = []
# for html_data in all_data:
#     my_end = re.search("//pp.walk.sc/badge/walk/score/", html_data).end()
#     walk_score.append(int(html_data[my_end:my_end+3].replace(".","").replace("p","")))
#     my_end = re.search("//pp.walk.sc/badge/bike/score/", html_data).end()
#     bike_score.append(int(html_data[my_end:my_end+3].replace(".","").replace("p","")))


In [139]:
# walk_score = pd.DataFrame({"city" : city,"state_code" : state_code,"walk_score" : walk_score, "bike_score" : bike_score})
# walk_score.to_csv("walk-score.csv", index = False)

In [140]:
walk_score = pd.read_csv("walk-score.csv")
walking_data = pd.merge(walking_data, walk_score, on = ["city", "state_code"], how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,mhlth,obesity,phlth,sleep,stroke,teethlost,cumulative_confirmed,cumulative_deceased,walk_score,bike_score
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,16.00,36.00,10.60,35.85,3.10,11.85,0.482257,0.004340,28,38
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,15.15,38.85,10.35,37.70,3.55,12.10,0.640776,0.008178,33,31
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,16.80,40.15,12.25,37.85,3.60,14.00,0.625683,0.008961,25,40
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,16.85,39.10,12.30,39.90,3.95,14.20,0.362487,0.007109,22,31
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,15.15,38.85,10.35,37.70,3.55,12.10,0.640776,0.008178,17,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,13.70,36.10,10.15,32.90,3.00,10.20,0.748493,0.008946,56,54
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,11.70,28.95,7.70,28.15,2.30,6.60,1.626347,0.015339,35,38
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,14.05,37.65,10.20,32.70,3.15,11.20,0.441196,0.003645,60,51
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,13.25,34.10,9.35,33.30,2.65,12.90,0.381547,0.004654,36,42


In [141]:
transportation = pd.read_csv("Means_of_Transportation_2021-Data.csv",encoding='latin-1')
col_name = ["NAME", "S0801_C01_002E", "S0801_C01_009E", "S0801_C01_010E","S0801_C01_011E","S0801_C01_013E"]
col_label = ["city_state", "drive_commute","public_transit_commute","walk_commute", "bike_commute", "work_from_home"]

transportation["NAME"] = [i.replace(" city","") for i in transportation["NAME"]]
transportation = transportation[col_name].set_axis(col_label, axis="columns", copy=False)
transportation = transportation.replace("N",float('nan'))

walking_data = pd.merge(walking_data, transportation, on = "city_state", how = "left")

walking_data

transportation

,city_state,drive_commute,public_transit_commute,walk_commute,bike_commute,work_from_home
0,"Auburn, Alabama",NaN,NaN,NaN,NaN,NaN
1,"Birmingham, Alabama",83.6,0.9,2.8,0.1,12
2,"Dothan, Alabama",89.1,0,1.3,0.1,7.9
3,"Hoover, Alabama",NaN,NaN,NaN,NaN,NaN
4,"Huntsville, Alabama",75.5,0.1,0.9,0.1,22.1
...,...,...,...,...,...,...
629,"Caguas zona urbana, Puerto Rico",NaN,NaN,NaN,NaN,NaN
630,"Carolina zona urbana, Puerto Rico",NaN,NaN,NaN,NaN,NaN
631,"Guaynabo zona urbana, Puerto Rico",NaN,NaN,NaN,NaN,NaN
632,"Ponce zona urbana, Puerto Rico",NaN,NaN,NaN,NaN,NaN


In [142]:
household = pd.read_csv("Households-2021-Data.csv",encoding='latin-1')
col_name = ["NAME","S1903_C01_001E"]
col_label = ["city_state", "households"]

household["NAME"] = [i.replace(" city","") for i in household["NAME"]]
household = household[col_name].set_axis(col_label, axis="columns", copy=False)
household = household.replace("N",float('nan'))
household
walking_data = pd.merge(walking_data, household, on = "city_state", how = "left")

walking_data["households"] = [float(house)/pop for house, pop in zip(walking_data["households"], walking_data["pop_estimate_2021"])]



In [143]:
income = pd.read_csv("Mean_Income_2021-Data.csv",encoding='latin-1')
col_name = ["NAME","S1902_C03_001E","S1902_C03_019E"]
col_label = ["city_state","mean_household_income","mean_income"]

income["NAME"] = [i.replace(" city","") for i in income["NAME"]]
income = income[col_name].set_axis(col_label, axis="columns", copy=False)
income = income.replace("N",float('nan'))
income.drop_duplicates(subset="city_state", inplace=True)
walking_data = pd.merge(walking_data, income, on = "city_state", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,walk_score,bike_score,drive_commute,public_transit_commute,walk_commute,bike_commute,work_from_home,households,mean_household_income,mean_income
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,28,38,NaN,NaN,NaN,NaN,NaN,0.370857,79325,29770
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,33,31,83.6,0.9,2.8,0.1,12,0.443224,58012,27211
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,25,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,22,31,89.1,0,1.3,0.1,7.9,0.425725,70513,31168
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,17,12,NaN,NaN,NaN,NaN,NaN,0.396602,124562,50046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,56,54,84,1.8,1.8,0,8.6,0.400443,60267,24591
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,35,38,79.2,0.7,2.3,0,17.2,0.437002,88781,38381
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,60,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,36,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
income = pd.read_csv("Median_Income_2021-Data.csv",encoding='latin-1')
col_name = ["NAME","S1903_C03_001E"]
col_label = ["city_state","median_household_income"]

income["NAME"] = [i.replace(" city","") for i in income["NAME"]]
income = income[col_name].set_axis(col_label, axis="columns", copy=False)
income = income.replace("N",float('nan'))
income.drop_duplicates(subset="city_state", inplace=True)

walking_data = pd.merge(walking_data, income, on = "city_state", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,bike_score,drive_commute,public_transit_commute,walk_commute,bike_commute,work_from_home,households,mean_household_income,mean_income,median_household_income
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,38,NaN,NaN,NaN,NaN,NaN,0.370857,79325,29770,48531
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,31,83.6,0.9,2.8,0.1,12,0.443224,58012,27211,36614
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,31,89.1,0,1.3,0.1,7.9,0.425725,70513,31168,45088
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,12,NaN,NaN,NaN,NaN,NaN,0.396602,124562,50046,99276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,54,84,1.8,1.8,0,8.6,0.400443,60267,24591,47861
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,38,79.2,0.7,2.3,0,17.2,0.437002,88781,38381,69533
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
# import requests
# import re
# states = pd.read_csv("state.csv",encoding='latin-1')
# states = states["state"]
# number = [i for i in range(1,100) if not i in [3,7,14,43,52]]

# county_state = []
# living_wage = []

# for idx, state in enumerate(states):
#     print(state)
    
#     url = "https://livingwage.mit.edu/states/"+str(number[idx])+"/locations"
#     print(url)
#     response = requests.get(url)
#     html_data = str(response.content)
#     my_start = re.search('<div class="counties list-unstyled">',html_data).start()
#     my_end = re.search('METROPOLITAN STATISTICAL AREAS',html_data).start()
#     html_data = html_data[my_start:my_end]

#     html_data = html_data.split("/counties/")
#     html_data = [i.split('">') for i in html_data if "County" in i]
#     county = [i[1].split("</a>\\n") for i in html_data]
#     county = [i[0].replace(" County","") for i in county]
#     county = [i+", "+ state for i in county]
#     county_state.extend(county)
#     html_data = [i[0] for i in html_data]
#     html_data
#     urls = ["https://livingwage.mit.edu/counties/" + i for i in html_data]
#     for url in urls:
#         response = requests.get(url)
#         html_data = str(response.content)
#         my_start = re.search('Required annual income before taxes', html_data).end()
#         my_end = re.search('Typical Annual Salaries',html_data).start()
#         html_data = html_data[my_start:my_end].replace("</td>\\n      <td>\\n        $","").replace(",","")
#         salary = int(html_data.split("\\")[0])
#         living_wage.append(salary)


# wage = pd.DataFrame({"county_state":county_state, "living_wage":living_wage})
# wage.to_csv("living_wage.csv",index=False)



In [146]:
wage = pd.read_csv("living_wage.csv",encoding='latin-1')
walking_data = pd.merge(walking_data, wage, on = ["county_state"], how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,drive_commute,public_transit_commute,walk_commute,bike_commute,work_from_home,households,mean_household_income,mean_income,median_household_income,living_wage
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,NaN,NaN,NaN,NaN,NaN,0.370857,79325,29770,48531,32556.0
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,83.6,0.9,2.8,0.1,12,0.443224,58012,27211,36614,36193.0
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31259.0
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,89.1,0,1.3,0.1,7.9,0.425725,70513,31168,45088,31803.0
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,NaN,NaN,NaN,NaN,NaN,0.396602,124562,50046,99276,36193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,84,1.8,1.8,0,8.6,0.400443,60267,24591,47861,33541.0
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,79.2,0.7,2.3,0,17.2,0.437002,88781,38381,69533,34586.0
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34586.0
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30785.0


In [147]:
# get data from counties, webscrape from MIT

# cost_of_living = pd.read_csv("cost_of_living_index.csv",encoding='latin-1')
# col_label = ["city", "state_code", "cost_of_living_index"]
# cost_of_living = cost_of_living.set_axis(col_label, axis="columns", copy=False)
# cost_of_living
# walking_data = pd.merge(walking_data, cost_of_living, on = ["city","state_code"], how = "left")


In [148]:
poverty = pd.read_csv("Poverty_2021-Data.csv",encoding='latin-1',low_memory=False)
col_name = ["NAME","S1701_C02_001E"]
col_label = ["city_state", "poverty"]
poverty = poverty[[i.__contains__("city") for i in poverty["NAME"]]]
poverty["NAME"] = [i.replace(" city","") for i in poverty["NAME"]]
poverty = poverty[col_name].set_axis(col_label, axis="columns", copy=False)
poverty = poverty.replace("N",float('nan'))
poverty.drop_duplicates(subset="city_state", inplace=True)

walking_data = pd.merge(walking_data, poverty, on = "city_state", how = "left")
walking_data["poverty"] = [pov/pop for pov, pop in zip(walking_data["poverty"],walking_data["pop_estimate_2021"])]
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,public_transit_commute,walk_commute,bike_commute,work_from_home,households,mean_household_income,mean_income,median_household_income,living_wage,poverty
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,NaN,NaN,NaN,NaN,0.370857,79325,29770,48531,32556.0,0.231926
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,0.9,2.8,0.1,12,0.443224,58012,27211,36614,36193.0,0.270712
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31259.0,NaN
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,0,1.3,0.1,7.9,0.425725,70513,31168,45088,31803.0,0.209933
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,NaN,NaN,NaN,NaN,0.396602,124562,50046,99276,36193.0,0.059813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,1.8,1.8,0,8.6,0.400443,60267,24591,47861,33541.0,0.221427
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,0.7,2.3,0,17.2,0.437002,88781,38381,69533,34586.0,0.077860
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34586.0,NaN
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30785.0,NaN


In [149]:
unemployment = pd.read_csv("Unemployment_2021-Data.csv",encoding='latin-1')
col_name = ["NAME","S2301_C04_001E"]
col_label = ["city_state","unemployment_rate"]

unemployment = unemployment[[i.__contains__("city") for i in unemployment["NAME"]]]
unemployment["NAME"] = [i.replace(" city","") for i in unemployment["NAME"]]
unemployment = unemployment[col_name].set_axis(col_label, axis="columns", copy=False)
unemployment = unemployment.replace("N",float('nan'))
unemployment.drop_duplicates(subset="city_state", inplace=True)

walking_data = pd.merge(walking_data, unemployment, on = "city_state", how = "left")
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,walk_commute,bike_commute,work_from_home,households,mean_household_income,mean_income,median_household_income,living_wage,poverty,unemployment_rate
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,NaN,NaN,NaN,0.370857,79325,29770,48531,32556.0,0.231926,NaN
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,2.8,0.1,12,0.443224,58012,27211,36614,36193.0,0.270712,7.4
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31259.0,NaN,NaN
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,1.3,0.1,7.9,0.425725,70513,31168,45088,31803.0,0.209933,5.3
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,NaN,NaN,NaN,0.396602,124562,50046,99276,36193.0,0.059813,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,1.8,0,8.6,0.400443,60267,24591,47861,33541.0,0.221427,6.4
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,2.3,0,17.2,0.437002,88781,38381,69533,34586.0,0.077860,NaN
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34586.0,NaN,NaN
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30785.0,NaN,NaN


In [150]:
aqi = pd.read_csv("annual_aqi_by_county_2021.csv",encoding='latin-1')
col_name = ["State", "County", "Median AQI"]
col_label = ["state", "county", "median_aqi"]

aqi = aqi[col_name].set_axis(col_label, axis="columns", copy=False)
aqi["county_state"] = aqi["county"]+", "+ aqi["state"]
aqi = aqi[["county_state", "median_aqi"]]

# aqi = aqi.replace("N",float('nan'))
# aqi.drop_duplicates(subset=["state", "county"], inplace=True)

walking_data = pd.merge(walking_data, aqi, on = ["county_state"], how = "left")
walking_data



,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,bike_commute,work_from_home,households,mean_household_income,mean_income,median_household_income,living_wage,poverty,unemployment_rate,median_aqi
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,NaN,NaN,0.370857,79325,29770,48531,32556.0,0.231926,NaN,NaN
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,0.1,12,0.443224,58012,27211,36614,36193.0,0.270712,7.4,51.0
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,NaN,NaN,NaN,NaN,NaN,NaN,31259.0,NaN,NaN,39.0
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,0.1,7.9,0.425725,70513,31168,45088,31803.0,0.209933,5.3,NaN
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,NaN,NaN,0.396602,124562,50046,99276,36193.0,0.059813,3.5,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,0,8.6,0.400443,60267,24591,47861,33541.0,0.221427,6.4,43.0
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,0,17.2,0.437002,88781,38381,69533,34586.0,0.077860,NaN,43.0
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,NaN,NaN,NaN,NaN,NaN,NaN,34586.0,NaN,NaN,45.0
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,NaN,NaN,NaN,NaN,NaN,NaN,30785.0,NaN,NaN,42.0


In [151]:
# import requests
# states = walking_data["state"].drop_duplicates()
# urls = ["".join(["https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-8/table-8-state-cuts/", state.lower().replace(" ","_"), ".xls/output.xls"]) for state in states]
# responses = [requests.get(url) for url in urls]
# files = ["".join([state.lower().replace(" ","_"),".xls"]) for state in states]
# for idx, response in enumerate(responses):
#     open(files[idx], "wb").write(response.content)


In [152]:
# states = walking_data["state"].drop_duplicates()
# files = ["".join([state.lower().replace(" ","_"),".xls"]) for state in states]
# col_name = ["City", "state", "Violent\ncrime", "Property\ncrime"]
# col_label = ["city", "state", "violent_crime", "property_crime"]
# crime = pd.DataFrame({"city":[], "state":[], "violent_crime":[], "property_crime":[]})
# states.reset_index(inplace = True, drop = True)

# for idx, my_file in enumerate(files):
#     crime_add = pd.read_excel(my_file, skiprows=4)
#     crime_add["state"] = states[idx]
#     crime_add = crime_add[col_name].set_axis(col_label, axis="columns", copy=False)
#     crime = pd.concat([crime,crime_add])

# crime.to_csv("Crime_2019.csv", index = False)
    


In [153]:
crime = pd.read_csv("Crime_2019.csv",encoding='latin-1')
walking_data = pd.merge(walking_data, crime, on = ["city","state"], how = "left")
walking_data["violent_crime"] = [vio/pop for vio, pop in zip(walking_data["violent_crime"], walking_data["pop_estimate_2021"])]
walking_data["property_crime"] = [prop/pop for prop, pop in zip(walking_data["property_crime"], walking_data["pop_estimate_2021"])]
walking_data

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,households,mean_household_income,mean_income,median_household_income,living_wage,poverty,unemployment_rate,median_aqi,violent_crime,property_crime
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076,157.084761,...,0.370857,79325,29770,48531,32556.0,0.231926,NaN,NaN,NaN,NaN
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000,380.801025,...,0.443224,58012,27211,36614,36193.0,0.270712,7.4,51.0,NaN,NaN
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104,140.785624,...,NaN,NaN,NaN,NaN,31259.0,NaN,NaN,39.0,NaN,NaN
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184,232.650562,...,0.425725,70513,31168,45088,31803.0,0.209933,5.3,NaN,NaN,NaN
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896,124.490626,...,0.396602,124562,50046,99276,36193.0,0.059813,3.5,51.0,0.001231,0.020758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000,40.080067,...,0.400443,60267,24591,47861,33541.0,0.221427,6.4,43.0,0.003773,0.012836
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250,66.102904,...,0.437002,88781,38381,69533,34586.0,0.077860,NaN,43.0,0.000940,0.009683
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300,29.477371,...,NaN,NaN,NaN,NaN,34586.0,NaN,NaN,45.0,0.003547,0.028344
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150,68.757037,...,NaN,NaN,NaN,NaN,30785.0,NaN,NaN,42.0,0.002881,0.028590


In [154]:
from collections import Counter

walking_data["walkable"] = "Car Dependent"
walkable_rows = walking_data["walk_score"] > 49
walkable = walking_data["walkable"]
walkable[walkable_rows] = "Walkable"
walking_data["walkable"] = walkable

walking_data["walkable_detail"] = "Very Car Dependent"

walkable_rows1 = walking_data["walk_score"] > 89
walkable_rows2 = (89 >= walking_data["walk_score"]) & (walking_data["walk_score"] > 69)
walkable_rows3 = (69 >= walking_data["walk_score"]) & (walking_data["walk_score"] > 49)
walkable_rows4 = (49 >= walking_data["walk_score"]) & (walking_data["walk_score"] > 24)

walkable_detail = walking_data["walkable_detail"]
walkable_detail[walkable_rows1] = "Extremely Walkable"
walkable_detail[walkable_rows2] = "Very Walkable"
walkable_detail[walkable_rows3] = "Somewhat Walkable"
walkable_detail[walkable_rows4] = "Car Dependent"
walking_data["walkable_detail"] = walkable_detail

walking_data["aqi_concern_level"] = "Good"
aqi_rows1 = (100 >= walking_data["median_aqi"]) & (walking_data["median_aqi"] > 50)
aqi_rows2 = (150 >= walking_data["median_aqi"]) & (walking_data["median_aqi"] > 100)
aqi_rows3 = (200 >= walking_data["median_aqi"]) & (walking_data["median_aqi"] > 150)
aqi_rows4 = (300 >= walking_data["median_aqi"]) & (walking_data["median_aqi"] > 200)
aqi_rows5 = (walking_data["median_aqi"] > 300)

aqi_level = walking_data["aqi_concern_level"]

aqi_level[aqi_rows1] = "Moderate"
aqi_level[aqi_rows2] = "Unhealthy for Sensitive Groups"
aqi_level[aqi_rows3] = "Unhealthy"
aqi_level[aqi_rows4] = "Very Unhealthy"
aqi_level[aqi_rows5] = "Hazardous"

walking_data["aqi_concern_level"] = aqi_level

Counter(walking_data["aqi_concern_level"])

C:\Users\chane\AppData\Local\Temp\ipykernel_9128\1705161699.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  walkable[walkable_rows] = "Walkable"
C:\Users\chane\AppData\Local\Temp\ipykernel_9128\1705161699.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  walkable_detail[walkable_rows1] = "Extremely Walkable"
C:\Users\chane\AppData\Local\Temp\ipykernel_9128\1705161699.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  walkable_

Counter({'Good': 610, 'Moderate': 139, 'Unhealthy for Sensitive Groups': 11})

Add more code here ^. Use variable "walking_data" add more variables to the dataset. The codes below will upload "walking_data" to the csv and output the results. Do NOT edit any previous code. Just add on.

In [155]:
walking_data.to_csv("walkable-cities.csv", index = False)

In [156]:

walkable_cities = pd.read_csv("walkable-cities.csv")
walkable_cities

,place_code,city,state,pop_estimate_2021,state_code,city_state,city_state_code,place_state_code,geo_code,land_area_sqkm,...,median_household_income,living_wage,poverty,unemployment_rate,median_aqi,violent_crime,property_crime,walkable,walkable_detail,aqi_concern_level
0,3076,Auburn,Alabama,78564,AL,"Auburn, Alabama","Auburn, AL",3076AL,103076.0,157.084761,...,48531.0,32556.0,0.231926,NaN,NaN,NaN,NaN,Car Dependent,Car Dependent,Good
1,7000,Birmingham,Alabama,197575,AL,"Birmingham, Alabama","Birmingham, AL",7000AL,107000.0,380.801025,...,36614.0,36193.0,0.270712,7.4,51.0,NaN,NaN,Car Dependent,Car Dependent,Moderate
2,20104,Decatur,Alabama,57804,AL,"Decatur, Alabama","Decatur, AL",20104AL,120104.0,140.785624,...,NaN,31259.0,NaN,NaN,39.0,NaN,NaN,Car Dependent,Car Dependent,Good
3,21184,Dothan,Alabama,71175,AL,"Dothan, Alabama","Dothan, AL",21184AL,121184.0,232.650562,...,45088.0,31803.0,0.209933,5.3,NaN,NaN,NaN,Car Dependent,Very Car Dependent,Good
4,35896,Hoover,Alabama,92589,AL,"Hoover, Alabama","Hoover, AL",35896AL,135896.0,124.490626,...,99276.0,36193.0,0.059813,3.5,51.0,0.001231,0.020758,Car Dependent,Very Car Dependent,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,66000,Racine,Wisconsin,77127,WI,"Racine, Wisconsin","Racine, WI",66000WI,5566000.0,40.080067,...,47861.0,33541.0,0.221427,6.4,43.0,0.003773,0.012836,Walkable,Somewhat Walkable,Good
756,84250,Waukesha,Wisconsin,71256,WI,"Waukesha, Wisconsin","Waukesha, WI",84250WI,5584250.0,66.102904,...,69533.0,34586.0,0.077860,NaN,43.0,0.000940,0.009683,Car Dependent,Car Dependent,Good
757,85300,West Allis,Wisconsin,59484,WI,"West Allis, Wisconsin","West Allis, WI",85300WI,5585300.0,29.477371,...,NaN,34586.0,NaN,NaN,45.0,0.003547,0.028344,Walkable,Somewhat Walkable,Good
758,13150,Casper,Wyoming,58656,WY,"Casper, Wyoming","Casper, WY",13150WY,5613150.0,68.757037,...,NaN,30785.0,NaN,NaN,42.0,0.002881,0.028590,Car Dependent,Car Dependent,Good
